# Diabetes AUTOML with TPOT, Local Mode

In [35]:
# Command to install the below commands
# !pip3 install tpot
# !pip3 install torch
!conda install -y -c conda-forge xgboost

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.0
  latest version: 4.9.2

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.



In [36]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import tpot
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import joblib




In [38]:
column_list_file = './Files/diabetes_train_column_list.txt'
train_file = './Files/diabetes_train.csv'
validation_file = './Files/diabetes_validation.csv'
entire_dataset = './Files/diabetes_train_all_normalized.csv'

## Extract Training, Validation files and Columns

In [39]:
with open(column_list_file,'r') as f:
    columns = f.read().split(',')
    
columns


['diabetes_class',
 'preg_count',
 'glucose_concentration',
 'diastolic_bp',
 'triceps_skin_fold_thickness',
 'two_hr_serum_insulin',
 'bmi',
 'diabetes_pedi',
 'age']

In [40]:

df_train = pd.read_csv(train_file, names=columns)
df_validation = pd.read_csv(validation_file,names=columns)

In [41]:
#Flatten target to 1D array with ravel()
X_train = df_train.iloc[:,1:]
y_train = df_train.iloc[:,0].ravel()

X_valid = df_validation.iloc[:,1:]
y_valid = df_validation.iloc[:,0].ravel()

In [42]:
import tpot
auto_ml = tpot.TPOTClassifier(generations=5,population_size=100,
         offspring_size=None,mutation_rate=0.9,crossover_rate=0.1,
         scoring=None,cv=5,subsample=1.0,n_jobs=1,
         max_time_mins=None,max_eval_time_mins=5,random_state=None,
         config_dict=None,template=None,warm_start=False,
         memory=None,use_dask=False,periodic_checkpoint_folder=None,
         early_stop=None,verbosity=2,
         disable_update_check=False)

auto_ml.fit(X_train,y_train)


Generation 1 - Current best internal CV score: 0.8993596400138456

Generation 2 - Current best internal CV score: 0.9011941848390446

Generation 3 - Current best internal CV score: 0.905019037729318

Generation 4 - Current best internal CV score: 0.905019037729318

Generation 5 - Current best internal CV score: 0.905019037729318

Best pipeline: XGBClassifier(input_matrix, learning_rate=1.0, max_depth=4, min_child_weight=14, n_estimators=100, nthread=1, subsample=0.9500000000000001)


TPOTClassifier(generations=5, verbosity=2)

In [43]:

# HBox(children=(FloatProgress(value=0.0, description='Optimization Progress', max=600.0, style=ProgressStyle(de…

# Generation 1 - Current best internal CV score: 0.8994115610938043

# Generation 2 - Current best internal CV score: 0.8994115610938043

# Generation 3 - Current best internal CV score: 0.9012634129456559

# Generation 4 - Current best internal CV score: 0.9031152647975077

# Generation 5 - Current best internal CV score: 0.9049844236760125

# Best pipeline: RandomForestClassifier(SelectFwe(input_matrix, alpha=0.041), bootstrap=False, criterion=entropy, max_features=0.4, min_samples_leaf=4, min_samples_split=5, n_estimators=100)
# TPOTClassifier(generations=5, verbosity=2)

In [44]:
clf = RandomForestClassifier(bootstrap=False, criterion='entropy', max_features=0.4, min_samples_leaf=4, min_samples_split=5, n_estimators=100)

clf.fit(X_train, y_train)
clf_results = clf.predict(X_valid)


In [31]:
print(classification_report(
    df_validation['diabetes_class'],
    clf_results,
    labels=[1,0],
    target_names=['Diabetic','Normal']))

              precision    recall  f1-score   support

    Diabetic       0.85      0.85      0.85        79
      Normal       0.92      0.92      0.92       152

    accuracy                           0.90       231
   macro avg       0.88      0.88      0.88       231
weighted avg       0.90      0.90      0.90       231



In [33]:
# add predicted to df_train and check confusion matrix
# check logloss training and validation error
# if everything is looks good on the test set, then re-train the model on entire dataset then serialize it using joblib

In [34]:
classifier = xgb.XGBClassifier(objective="binary:logistic")
classifier

NameError: name 'xgb' is not defined

In [ ]:
classifier.fit(X_train,
               y_train, 
               eval_set = [(X_train, y_train), (X_valid, y_valid)], 
               eval_metric=['logloss'],
               early_stopping_rounds= 20)


In [ ]:
classifier.fit(X_train,
               y_train, 
               eval_set = [(X_train, y_train), (X_valid, y_valid)], 
               eval_metric=['logloss'],
               early_stopping_rounds= 20)
